## Used files

In [2]:
!ls -lh w2.txt w5.txt reviews.csv big.txt

-rwxrwxrwx 1 root root 6.2M Feb 28 13:40 big.txt
-rwxrwxrwx 1 root root  64M Oct 19  2019 reviews.csv
-rwxrwxrwx 1 root root  17M Feb 29 11:51 w2.txt
-rwxrwxrwx 1 root root  27M Feb 28 19:33 w5.txt


# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

### Norvig's solution

In [3]:
import re
from collections import Counter
import pandas as pd
import numpy as np

def words(text): return re.findall(r'\w+', text.lower())

file = open('big.txt').read()
WORDS = Counter(words(file))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]).union(known(edits1(word))) or known(edits2(word)) or [word])

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

# My solution

### Import ngrams data

In [4]:
n5 = {}
n3 = {}
n4 = {}
# Read 5-gram file and store n-grams in dictionaries
# And update 4-gram and 3-gram dictionaries along with 5-gram
with open('w5.txt', 'r') as file:
    lines = file.readlines()
    for i in lines:
        # Split line into count and n-gram
        i = i.strip().split("\t")

        # Identify n-grams
        gram5 = tuple(i[1:])
        gram4 = [gram5[1:], gram5[:-1]]
        gram3 = [gram5[1:4], gram5[2:], gram5[:-2]]
        
        count = int(i[0])

        # Store 5-gram in dictionary without duplicates
        if gram5 not in n5:
            n5[gram5] = count
        else: raise Exception("Duplicate ngram")


        # Update 4-gram dictionary
        for ngram in gram4:
            if ngram not in n4:
                n4[ngram] = count
            else: 
                n4[ngram] += count

        # Update 3-gram dictionary
        for ngram in gram3:
            if ngram not in n3:
                n3[ngram] = count
            else: 
                n3[ngram] += count
            

n2 = {}
# Read 2-gram file
with open('w2.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for i in lines:
        i = i.strip().split("\t")
        ngram = tuple(i[1:])
        count = int(i[0])

        # Store 2-gram in dictionary without duplicates
        if ngram not in n2:
            n2[ngram] = count
        else: raise Exception("Duplicate ngram")

### Check imported data

In [5]:
grams = [n2, n3, n4, n5]
names = ["2-gram", "3-gram", "4-gram", "5-gram"]

for i in range(4):
    print(names[i], ":", len(grams[i]))
    print("Most common n-grams:")
    print(Counter(grams[i]).most_common(5))
    print()

2-gram : 1020368
Most common n-grams:
[(('of', 'the'), 2586813), (('in', 'the'), 2043262), (('to', 'the'), 1055301), (('on', 'the'), 920079), (('and', 'the'), 737714)]

3-gram : 619624
Most common n-grams:
[(('i', 'do', "n't"), 193377), (('a', 'lot', 'of'), 167920), (('one', 'of', 'the'), 159272), (('the', 'united', 'states'), 144059), (('do', "n't", 'know'), 113219)]

4-gram : 999232
Most common n-grams:
[(('the', 'end', 'of', 'the'), 53263), (('do', "n't", 'want', 'to'), 50903), (('i', 'do', "n't", 'think'), 50304), (('i', 'do', "n't", 'know'), 42473), (('in', 'the', 'united', 'states'), 39050)]

5-gram : 1044268
Most common n-grams:
[(('at', 'the', 'end', 'of', 'the'), 13588), (('i', 'do', "n't", 'want', 'to'), 12744), (('in', 'the', 'middle', 'of', 'the'), 9124), (('i', 'do', "n't", 'know', 'what'), 8076), (('you', 'do', "n't", 'have', 'to'), 7186)]



### Functions to consider context of word

In [6]:
def find_by_context(word, context):
    '''Find the word in the ngram context
    Returns the context length and the count of the word occurences for each found ngram contexts
    First context length are longer
    context: list of words before the word to find'''
    context = context[-4:] # for maximum 5-gram

    found_contexts = []
    ns = [None, n2, n3, n4, n5]
    for context_len in range(len(context), 0, -1):
        # ngram to number of occurences
        ngrams = ns[context_len]
        context = context[-context_len:]

        # find the word in the ngram context
        # return context len and the count of the word occurences in such ngram
        if (*context, word) in ngrams:
            # compact the context count
            if (len(found_contexts) > 0 and found_contexts[-1][0] == context_len):
                found_contexts[-1] = (context_len, found_contexts[-1][1] + ngrams[(*context, word)])
            else:
                found_contexts.append((context_len, ngrams[(*context, word)]))
    
    return found_contexts    

In [7]:
from functools import cmp_to_key

# comparator for the context
def compare_contexts(c1, c2):
    '''Compare two contexts
    Return 0 if they are equal
    Return >0 if c2 has longer context length or higher count
    '''

    if (len(c1) == 0 and len(c2) == 0):
        return 0
    elif len(c1) == 0:
        return -1
    elif len(c2) == 0:
        return 1
    
    # compare by context length
    if c1[0][0] != c2[0][0]:
        return c1[0][0] - c2[0][0]
    # compare by count
    return c1[0][1] - c2[0][1]

def find_max_contex_index(contexts):
    '''Find the max context in the list of contexts'''
    max_context = max(contexts, key=cmp_to_key(compare_contexts))
    # find index of the max context
    max_context_index = contexts.index(max_context)
    return max_context_index

# Test the function
c1 = find_by_context("the", ['is'])
c2 = find_by_context("he", ['is'])
c3 = find_by_context("the", ['is','of'])
c4 = find_by_context("the", ['is', 'of', 'the'])
c = [c1, c2, c3, c4]
print(c)
print(find_max_contex_index(c))

[[(1, 275367)], [(1, 11787)], [(2, 289), (1, 2586813)], [(1, 1927)]]
2


In [36]:
def correct_by_context(wrong, context, verbose=False):
    '''Correct the word by the context
    for each candidate search for the context and choose the one with the longest context or the highest count
    '''
    if verbose:
        print()
        print('Wrong word:', wrong)

    cands = list(candidates(wrong))
    found_contexts = []
    for c in cands:
        found_contexts.append(find_by_context(c, context))

    if all([len(f) == 0 for f in found_contexts]): 
        if verbose: print('No found context. Get most probable correction')
        return max(cands, key=P)
        
    best_context_index = find_max_contex_index(found_contexts)
    if verbose:
        print("Candidates:", cands)
        print("Best context index:", best_context_index)
        best_context = found_contexts[best_context_index]
        if (len(best_context) > 0):
            best_context = best_context[0]
            print(f'Top context len: {best_context[0]} with occurences {best_context[1]}')
    return cands[best_context_index]

# Test the function
print(correct_by_context("e", words('He could'), verbose=True))
print(correct_by_context("he", words("with the help of"), verbose=True)) # good context
print(correct_by_context("yo", words('I want'), verbose=True)) # easy context
print(correct_by_context("yo", words('I state I love'), verbose=True)) # to be 2 context
print(correct_by_context("yo", words('I think I love'), verbose=True)) # 4 context
print(correct_by_context("he", words('How this film could'), verbose=True)) # good context
print(correct_by_context("he", words('I am going to a back room of'), verbose=True)) # long context (will be cut)
print(correct_by_context("he", words(''), verbose=True)) # No context


Wrong word: e
Candidates: ['ex', 'es', 'em', 'er', 'oe', 'ed', 'w', 'eh', 'o', 'le', 'a', 're', 've', 'be', 'd', 'te', 'ne', 'z', 'ye', 'c', 'he', 'j', 'u', 'ce', 'p', 'en', 'se', 'fe', 'et', 'el', 'eg', 'm', 'k', 'b', 'i', 'q', 't', 'ke', 's', 'e', 'me', 'we', 'v', 'ev', 'g', 'f', 'n', 'de', 'y', 'ze', 'x', 'je', 'pe', 'l', 'r', 'h']
Best context index: 13
Top context len: 2 with occurences 613
be

Wrong word: he
Candidates: ['hen', 'oe', 'eh', 'hem', 'le', 're', 'hue', 'be', 've', 'te', 'ne', 'ye', 'hey', 'he', 'her', 'ce', 'se', 'she', 'fe', 'ha', 'hi', 'hm', 'ho', 'the', 'ke', 'e', 'me', 'we', 'heh', 'de', 'ze', 'je', 'pe', 'hew', 'h']
Best context index: 23
Top context len: 4 with occurences 564
the

Wrong word: yo
Candidates: ['yon', 'vo', 'o', 'to', 'co', 'go', 'no', 'mo', 'wo', 'you', 'do', 'myo', 'ye', 'lo', 'oo', 'po', 'so', 'y', 'fo', 'ho']
Best context index: 3
Top context len: 2 with occurences 39398
to

Wrong word: yo
Candidates: ['yon', 'vo', 'o', 'to', 'co', 'go', 'no'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

1. Firstly, instead of returning candidates if the word is known, I return this word along with one edit distance candidates instead of returning nothing as Norvig. This is because the word might be known, but the word in the context is different. For example, "he" is known, but in the context "he needs" it might be "the needs". So, 'he' will always be checked as 'me', 'be', 'the' etc.
2. Secondly, I use suggested datasets of 2 and 5 grams since it has over 1M ngrams. 
3. Then, I construct 3 and 4 grams datasets from 5grams by slicing them. I do it also for performance for fast search of 3gram context instead of all 5grams. That's why all my corrections have the same worst case time complexity as Norvig.
4. For each candidate word I check the presence of it with some context in my dictionaries. As a final decision which word to suggest I base on the following:
    1. The longer the context of a word the higher chances for it to be suggested. I do it since it is barely possible to overfit with the data, and long custructions such as 'with the help of' should always be prefered to smaller contexts. And context of len 1 is always prefered to popular suggestion but without contex coupling
    2. If 2 candidate words have equal largest contexts I just look at their occurances as the higher it is the more probably in my data the word will appear
5. In addition, I do not find separate contexts occurances of the same lengths. For a word I combine all the same length context into single tuple (context_len, n) where n is total count of occurances in the contexts with length context_len. So, the more a word appear in long contexts the more probable it is to be "typed" by a user as they want


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [23]:
# IMDB reviews dataset of sentences
reviews = pd.read_csv("reviews.csv")['review'].apply(lambda t: t.lower()).apply(words)
reviews = [r[:5] for r in reviews if r[4] != 'br'] # remove <br> tags
reviews = reviews[:5000]

In [24]:
# a function that takes a word and returns it with some error or noise
def mutate_word(word):
    '''applies some mutation on a word
    random letter replacement
    removal of a random letter if there are at least 2
    insertion of a letter at the end
    '''
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    action = np.random.randint(0, 3)
    
    if action == 0:
        # replace a letter
        word = list(word)
        word[np.random.randint(0, len(word))] = alphabet[np.random.randint(0, 26)]
        word = ''.join(word)
    elif action == 1 and len(word) > 1:
        # remove a letter
        rem_index = np.random.randint(0, len(word))
        word = word[:rem_index] + word[rem_index+1:]
    else:
        # add a letter
        word = word + alphabet[np.random.randint(0, 26)]

    return word

### Test Norvig's corrector

In [44]:
import time

# last word is incorrect in each sentence
test_sentences = [r[:-1] + [mutate_word(r[-1])] for r in reviews]
good = 0
unknown = 0
n = len(test_sentences)
verbose = False

start = time.time()
for sent, corr_sent in zip(test_sentences,reviews):
    wrong = sent[-1]
    corr = corr_sent[-1]
    pred = correction(wrong)
    unknown += (corr not in WORDS)
    good += (pred == corr)
    if verbose and pred != corr:
        print('correction({}) => {} ({}); expected {} ({}). Context: {}'.format(wrong, pred, WORDS[pred], corr, WORDS[corr], " ".join(sent)))

dt = time.time() - start

print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '.format(good / n, n, unknown / n, n / dt))

64% of 5000 correct (8% unknown) at 269 words per second 


### Test my corrector

In [37]:
good = 0
unknown = 0

verbose = False

start = time.time()
for sent, corr_sent in zip(test_sentences,reviews):
    wrong = sent[-1]
    context = sent[:-1]
    corr = corr_sent[-1]
    pred = correct_by_context(wrong, context)
    unknown += (corr not in WORDS)
    good += (pred == corr)
    if verbose and pred != corr:
        print('correction({}) => {}; expected {}. Context: {}'.format(wrong, pred, corr, " ".join(sent)))

dt = time.time() - start
print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '.format(good / n, n, unknown / n, n / dt))

76% of 5000 correct (8% unknown) at 273 words per second 


In [41]:
# Straight comparison of correction for word
word = 'ove'
context_sent = words('you are my')

# Norvig function vs My function
correction(word), correct_by_context(word, context_sent)

('one', 'love')

### Comparison and accuracies
- My solution: 76% accuracy. It is definitely not worse since if no context was familiar it will produce most common correction as Norvig. In other cases it checks for at most 4 context types (2,3,4,5-grams) in dictionaries which is not slower as well.
- Norvig's solution: 64% accuracy. Does not use any context, sometimes it is even a bit slower on my machine ^_^ than my solution with ngram context since the complexity is nearly the same